In [1]:
import random 
import copy
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

class Inputdata:
    def __init__(self, test_dict, test_id_machine_dict, test_id_time_dict, test_id_resource_dict):
        self.test_dict = test_dict
        self.test_id_machine_dict = test_id_machine_dict
        self.test_id_time_dict = test_id_time_dict
        self.test_id_resource_dict = test_id_resource_dict
        
class Operation:
    def __init__(self, operationid, job, machine, jobprocessor, jobsuccessor, time, testid):
        self.oid = operationid
        self.j = job
        self.m = machine
        self.jp = jobprocessor
        self.js = jobsuccessor
        self.p = time
        self.testid = testid
        self.mp = None
        self.ms = None
        self.starttime = None
        self.endtime = None
        self.followby = None
        self.iscritical = False
        self.r = 0
        self.q = 0
        
class Job:
    def __init__(self, jobid):
        self.jid = jobid
        
class Machine:
    def __init__(self, machineid):
        self.mid = machineid
        
class Problem:
    def __init__(self, Olist, Mlist, Jlist):
        self.m = len(Mlist)
        self.n = len(Jlist)
        self.operations = Olist
        self.machines = Mlist
        self.jobs = Jlist
        
class Solution:
    def __init__(self, problem, schedule):
        self.prob = problem
        self.sche = schedule
        self.cost = None
        self.topoorder = None
        self.criticalpath = None
        
class Swap:
    def __init__(self, operation1, operation2):
        self.oper1 = operation1
        self.oper2 = operation2
    
    def equals(self, swap):
        if(self.oper1 == swap.oper1 and self.oper2 == swap.oper2):
            return True
        return False

class Insert:
    def __init__(self, sequence, typ):
        self.seq = sequence
        self.type = typ
        self.forseq = [sequence[-1]] + sequence[0:-1]
        self.backseq = sequence[1:] + [sequence[0]]
    
    def printmove(self):
        print('{', end = "")
        for oper in self.seq:
            print(str(oper.oid) + " ", end = "")
        print(self.type + "}")
    

class Parameter:
    def __init__(self, maxiter, maximproveiter):
        self.maxiter = maxiter
        self.maximproveiter = maximproveiter
        
class Tabulist:
    def __init__(self, listlength):
        self.list = []
        self.listlength = listlength
    
    def addmove(self, move, option):
        if(not self.containmove(move, option)):
            if(option == "N5"):
                mov = Swap(move.oper2, move.oper1)
                self.list.append(mov)
                if(len(self.list) > self.listlength):
                    self.list.remove(self.list[0])
            elif(option == "N6"):
                self.list.append(move.seq)
                if(len(self.list) > self.listlength):
                    self.list.remove(self.list[0])
    
    def containmove(self, move, option):
        if(option == "N5"):
            for mov in self.list:
                if mov.equals(move):
                    return True
            return False
        elif(option == "N6"):
            for seq in self.list:
                if (move.type == "F" and move.forseq == seq):
                    return True
                elif (move.type == "B" and move.backseq == seq):
                    return True
            return False
        
    def printtabulist(self):
        print("***tabu**")
        for seq in self.list:
            print("{", end = "")
            for oper in seq:
                print(oper.oid, end = " ")
            print("}")
        print("*********")
        
    
def stopcriteria(param, itera, improveiter, isoptimal):
    if itera >= param.maxiter or improveiter > param.maximproveiter or isoptimal:
        return True
    return False
       
def implementmove(sol, move, option):
    if(move == None):
        return sol
    elif(option == 'N5'):
        return implementswap(sol, move)
    elif(option == "N6"):
        return implementinsert(sol, move)

def implementswap(sol, swap):
    oper1 = swap.oper1
    oper2 = swap.oper2
    mid = oper1.m.mid
    k1 = None
    k = 0
    while(True):
        if(sol.sche[mid][k] == oper1.oid):
            k1 = k
            break
        else:
            k = k + 1
    sol.sche[mid][k1] = oper2.oid
    sol.sche[mid][k1 + 1] = oper1.oid
    oper2.mp = oper1.mp
    oper1.ms = oper2.ms
    oper1.mp = oper2
    oper2.ms = oper1
    setgraph(sol)
    calculatetime(sol)
    findcriticalpath(sol)
    return sol

def implementinsert(sol, insert):
    sequence = insert.seq
    oper1 = sequence[0]
    mid = oper1.m.mid
    k1 = None
    k = 0
    while(True):
        if(sol.sche[mid][k] == oper1.oid):
            k1 = k
            break
        else:
            k = k + 1
    if(insert.type == "F"):
        for i in range(len(sequence)):
            sol.sche[mid][k1 + i] = insert.forseq[i].oid
        sequence[-1].ms.mp = sequence[-2]
        sequence[-2].ms = sequence[-1].ms
        sequence[0].mp.ms = sequence[-1]
        sequence[-1].mp = sequence[0].mp
        sequence[0].mp = sequence[-1]
        sequence[-1].ms = sequence[0]
    elif(insert.type == "B"):
        for i in range(len(sequence)):
            sol.sche[mid][k1 + i] = insert.backseq[i].oid
        sequence[-1].ms.mp = sequence[-2]
        sequence[0].mp.ms = sequence[1]
        sequence[1].mp = sequence[0].mp
        sequence[-1].ms.mp = sequence[0]
        sequence[0].ms = sequence[-1].ms
        sequence[-1].ms = sequence[0]
        sequence[0].mp = sequence[-1]
    setgraph(sol)
    calculatetime(sol)
    findcriticalpath(sol)
    return sol
            
            
def isconnected(sol, oper1, oper2):
    inx1 = sol.topoorder.index(oper1)
    inx2 = sol.topoorder.index(oper2)
    if inx1 == inx2:
        return True
    elif inx1 > inx2:
        return False
    else:
        return (isconnected(sol, oper1.js, oper2) or isconnected(sol, oper1.ms, oper2))

def readdata(scheduling_filename, matrix_filename, time_filename):
    with open(scheduling_filename) as f:
        lines = f.readlines()
    test_dict = {}
    for id, line in enumerate(lines):
        if line.strip():
            test_dict[id] = [x for x in line[:-1].split("->")]    
    df = pd.read_csv(matrix_filename, names = ["Components", "Test_id", "Resource", "0", "1", "2", "3", "4", "5"], skiprows=1)
    test_id_machine_dict = {}
    test_id_resource_dict = {}
    for index, row in df.iterrows():
        if not math.isnan(row["Test_id"]):
            test_id_machine_dict[str(int(row["Test_id"]))] = int(''.join([name for name in df.columns if row[name] == "Y"]))
            test_id_resource_dict[str(int(row["Test_id"]))] = int(row["Resource"])
    df = pd.read_csv(time_filename, names = ["Test_id", "Time"], skiprows=1)
    test_id_time_dict = {}
    for index, row in df.iterrows():
        if not math.isnan(row["Test_id"]):
            test_id_time_dict[str(int(row["Test_id"]))] = row["Time"]
    return Inputdata(test_dict,test_id_machine_dict,test_id_time_dict,test_id_resource_dict)

def generateproblem(inputdata):
    probn = len(inputdata.test_dict)
    probm = len(set(inputdata.test_id_machine_dict.values()))
    Olist = []
    Mlist = []
    Jlist = []
    index = 0
    Olist.append(Operation(index, None, None, None, None, 0, "Dummy"))
    index = index + 1
    for i in range(probn):
        Jlist.append(Job(i))
    for j in range(probm):
        Mlist.append(Machine(j))
    for i in inputdata.test_dict.keys():
        for test_id in inputdata.test_dict[i]:
            o = Operation(index, Jlist[i], Mlist[inputdata.test_id_machine_dict[test_id]], None, None, inputdata.test_id_time_dict[test_id], test_id)
            Olist.append(o)
            index = index + 1
    Olist.append(Operation(probn * probm + 1, None, None, None, None, 0, "Dummy"))
    for k in range(probn * probm):
        if(Olist[k + 1].j == Olist[k].j):
            Olist[k + 1].jp = Olist[k]
        else:
             Olist[k + 1].jp = Olist[0]
        if(Olist[k + 1].j == Olist[k + 2].j):
            Olist[k + 1].js = Olist[k + 2]
        else:
             Olist[k + 1].js = Olist[-1]
    return Problem(Olist, Mlist, Jlist)

def getinitialsche(prob):
    SetL = set();
    SetT = set();
    SetR = set();
    SetS = set();
    scheduling = [[None for col in range(prob.n)] for row in range(prob.m)]
    for oper in prob.operations[1:-1]:
        if oper.jp.oid == 0:
            oper.r = 0
            SetR.add(oper)
        elif oper.js.oid == prob.m * prob.n + 1:
            oper.q = 0
            SetS.add(oper)
    while(len(SetL) + len(SetT) < prob.m * prob.n):
        tmp = float("inf")
        if(len(SetR) != 0):
            for oper in SetR:
                if oper.r < tmp:
                    operation = oper
                    tmp = oper.r
            SetL.add(operation)
            SetR.remove(operation)
            SetS.discard(operation)
            i = 0
            while(scheduling[operation.m.mid][i] != None):
                i = i + 1
            scheduling[operation.m.mid][i] = operation.oid
            if operation.js not in SetT and operation.js != prob.operations[-1]:
                SetR.add(operation.js)
                operation.js.r = operation.r + operation.p
            for oper in SetR:
                if oper.m == operation.m and oper.r < operation.r + operation.p:
                    oper.r = operation.r + operation.p
        tmp = float("inf")
        if(len(SetS) != 0):
            for oper in SetS:
                if oper.q < tmp:
                    operation = oper
                    tmp = oper.q
            SetT.add(operation)
            i = -1
            while(scheduling[operation.m.mid][i] != None):
                i = i - 1
            scheduling[operation.m.mid][i] = operation.oid
            SetS.remove(operation)
            SetR.discard(operation)
            if operation.jp not in SetL and operation.jp != prob.operations[0]:
                SetS.add(operation.jp)
                operation.jp.q = operation.q + operation.p
            for oper in SetS:
                if oper.m == operation.m and oper.q < operation.q + operation.p:
                    oper.q = operation.q + operation.p
    return scheduling

def getinitialsol(prob, scheduling):
    sol = Solution(prob, scheduling)
    setgraph(sol)
    calculatetime(sol)
    findcriticalpath(sol)
    return sol

def setgraph(sol):
    for i in range(prob.m):
        prob.operations[sol.sche[i][0]].mp = prob.operations[0]
        prob.operations[sol.sche[i][0]].ms = prob.operations[sol.sche[i][1]]
        prob.operations[sol.sche[i][prob.n - 1]].mp = prob.operations[sol.sche[i][prob.n - 2]]
        prob.operations[sol.sche[i][prob.n - 1]].ms = prob.operations[-1]
        for j in range(prob.n - 2):
            prob.operations[sol.sche[i][j + 1]].mp = prob.operations[sol.sche[i][j]]
            prob.operations[sol.sche[i][j + 1]].ms = prob.operations[sol.sche[i][j + 2]]

def toposort(sol):
    visited = [False] * len(sol.prob.operations)
    stack = [sol.prob.operations[-1]]
    visited[-1] = True
    for oper in prob.operations[1:-1]:
        if visited[oper.oid] == False:
            toposortuntil(sol, oper, visited, stack)
    stack.insert(0, sol.prob.operations[0])
    sol.topoorder = stack
    return stack
    
def toposortuntil(sol, oper, visited, stack):
    visited[oper.oid] = True
    for v in [oper.js, oper.ms]:
        if visited[v.oid] == False:
            toposortuntil(sol, v, visited, stack)
    stack.insert(0, oper)
    
def calculatetime(sol):
    cost = 0
    last = None
    topostack = toposort(sol)
    sol.prob.operations[0].starttime = 0;
    sol.prob.operations[0].endtime = 0;
    for oper in topostack[1:-1]:
        if oper.jp.endtime >= oper.mp.endtime:
            oper.starttime = oper.jp.endtime
            oper.followby = oper.jp
        else: 
            oper.starttime = oper.mp.endtime
            oper.followby = oper.mp
        oper.endtime = oper.starttime + oper.p
        oper.r = oper.endtime
        if(oper.endtime > cost):
            cost = oper.endtime
            last = oper
        #print(oper.oid, oper.starttime, oper.endtime, oper.jp.oid, oper.jp.endtime, oper.mp.oid, oper.mp.endtime)
    sol.prob.operations[-1].starttime = cost
    sol.prob.operations[-1].endtime = cost
    sol.prob.operations[-1].followby = last
    sol.cost = cost
    sol.prob.operations[0].r = 0
    sol.prob.operations[0].q = sol.cost
    sol.prob.operations[-1].r = sol.cost
    sol.prob.operations[-1].q = 0
    for oper in topostack[-2:0:-1]:
        oper.q = max(oper.js.q + oper.js.p, oper.ms.q + oper.ms.p)
    findcriticalpath(sol)
    
def findcriticalpath(sol):
    oper = sol.prob.operations[-1]
    criticalpath = [[oper]]
    criticalblock = []
    tmp = 0
    while(oper.oid != sol.prob.operations[0].oid):
        oper = oper.followby
        criticalblock.insert(0, oper)
        if oper.followby == None or oper.followby.m != oper.m:
            criticalpath.insert(0, criticalblock)
            criticalblock = []
    sol.criticalpath = criticalpath

def findneighbors(sol, option):
    neighbors = []
    if(option == "N6"):
        for block in sol.criticalpath:
            if(len(block) >= 2):
                for i in range(len(block) - 1):
                    oper1 = block[0]
                    oper2 = block[i + 1]
                    if(oper1.r >= oper2.jp.r):
                        neighbors.append(Insert(block[0: i + 2], "F"))
                for i in range(len(block) - 1):
                    oper1 = block[i]
                    oper2 = block[-1]
                    if(oper2.q + oper2.p >= oper1.js.q + oper1.js.p):
                        neighbors.append(Insert(block[i:], "B")) 
    elif(option == "N5"):
        for block in sol.criticalpath:
            if(len(block) == 2):
                neighbors.append(Swap(block[0],block[1]))
            elif(len(block) > 2):
                neighbors.append(Swap(block[0],block[1]))
                neighbors.append(Swap(block[-2], block[-1]))
    return neighbors

def optimize(prob, initsche, method, neighboroption):
    param = Parameter(100000, 10000)
    if(method == "TS"):
        bestsol = tabusearch(prob, initsche, neighboroption, param)
    elif(method == "SA"):
        bestsol = simulatedannealing(prob, initsche, neighboroption, param)
    print(method + "&" + neighboroption + ":" + str(bestsol.cost))
    return bestsol
    

def tabusearch(prob, initsche, option, param):
    itera = 0
    im_iter = 0
    curr_sol = getinitialsol(prob, initsche)
    best_sol = curr_sol
    tabu = Tabulist(10)
    bestcost = curr_sol.cost
    isoptimal = False
    while(not stopcriteria(param, itera, im_iter, isoptimal)):
        itera = itera + 1
        im_iter = im_iter + 1
        neighbors = findneighbors(curr_sol, option)
        tmp = float("inf")
        optmove = None
        if(len(neighbors) == 0):
            isoptimal = True
        else:
            for move in neighbors:
                cost = estimatecost(curr_sol, move, option)
                if cost < tmp and (cost < bestcost or not tabu.containmove(move, option)):
                    tmp = cost
                    optmove = move
            if(optmove == None):
                optmove = random.choice(neighbors)
            #optmove.printmove()
            tabu.addmove(optmove, option)
            tmp = estimatecost(curr_sol, optmove, option)
            oldcost = curr_sol.cost
            curr_sol = implementmove(curr_sol, optmove, option)
            #printsche(curr_sol)
            #tabu.printtabulist()
            if(curr_sol.cost < bestcost):
                bestcost = curr_sol.cost
                best_sol = curr_sol
                im_iter = 0
    return best_sol

def Metropolis(new_cost, old_cost, T):
    if(T <= 1/(3 * (np.log(10)))):
        if(old_cost >= new_cost):
            return 1
        else:
            return 0
    else:
        return min(1, np.exp((old_cost - new_cost)/T))
    

def simulatedannealing(prob, initsche, option, param):
    itera = 0
    im_iter = 0
    curr_sol = getinitialsol(prob, initsche)
    best_sol = curr_sol
    T = 100
    bestcost = curr_sol.cost
    isoptimal = False
    while(not stopcriteria(param, itera, im_iter, isoptimal)):
        itera = itera + 1
        im_iter = im_iter + 1
        neighbors = findneighbors(curr_sol, option)
        tmp = float("inf")
        optmove = None
        oldcost = curr_sol.cost
        distribution = np.array([])
        if(len(neighbors) == 0):
            isoptimal = True
        else:
            for move in neighbors:
                cost = estimatecost(curr_sol, move, option)
                prob = Metropolis(cost, oldcost, T)/len(neighbors)
                if(prob <= 0.001):
                    prob = 0
                distribution = np.append(distribution, max(prob, 0))
            if(sum(distribution) > 1):
                distribution = distribution/sum(distribution)
            distribution = np.append(distribution, max(1 - sum(distribution), 0))
            neighbors.append(None)
            optmove = np.random.choice(neighbors, p = distribution)
            curr_sol = implementmove(curr_sol, optmove, option)
            if(curr_sol.cost < bestcost):
                bestcost = curr_sol.cost
                best_sol = curr_sol
                im_iter = 0
        T = T * 0.999
    return best_sol

def estimatecost(sol, mov, option):
    if(option == "N5"):
        oper1 = mov.oper1
        oper2 = mov.oper2
        delta = max(oper2.jp.r, oper1.mp.r) + oper2.p
        LB1 = delta + oper2.js.q + oper2.js.p
        LB2 = max(delta, oper1.jp.r) + oper1.p + max(oper1.js.q + oper1.js.p, oper2.ms.q + oper2.ms.p)
        LB = max(LB1, LB2)
        if LB >= sol.cost:
            return LB
        else:
            RQ = 0
            RQprime = 0
            beforeoperations = set()
            afteroper1 = False
            beforeoperations.add(prob.operations[0])
            for oper in sol.topoorder[1:-1]:
                if oper == oper1:
                    afteroper1 = True
                elif(not afteroper1):
                    beforeoperations.add(oper)
                    if(oper.js == prob.operations[-1] and oper.r > RQprime):
                        RQprime = oper.r
                    if (oper.ms == prob.operations[-1] and oper.r > RQprime):
                        RQprime = oper.r
                elif(afteroper1):
                    if(oper.jp == prob.operations[0] and oper.q + oper.p > RQprime):
                        RQprime = oper.q  + oper.p
                    if(oper.jp in beforeoperations and oper.jp.r + oper.q  + oper.p > RQ):
                        RQ = oper.jp.r + oper.q  + oper.p
                    if(oper.mp in beforeoperations and oper.mp.r + oper.q + oper.p > RQ):
                        RQ = oper.mp.r + oper.q  + oper.p
            return max(LB, RQ, RQprime)
    if(option == "N6"):
        sequence = mov.seq
        lambda0 = [0] * len(sequence)
        lambdan = [0] * len(sequence)
        if(mov.type == "B"):
            for i in range(len(sequence) - 1):
                if(i == 0):
                    lambda0[i + 1] = max(sequence[i + 1].jp.r, sequence[i].mp.r) 
                else:
                    lambda0[i + 1] = max(sequence[i + 1].jp.r, lambda0[i] + sequence[i].p)
            lambda0[0] = max(sequence[0].jp.r, lambda0[-1] + sequence[-1].p)
            lambdan[0] = sequence[0].p + max(sequence[0].js.q + sequence[0].js.p, sequence[-1].ms.q + sequence[-1].ms.p)
            lambdan[-1] = sequence[-1].p + max(sequence[-1].js.q + sequence[-1].js.p, lambdan[0])
            for i in range(len(sequence) - 2, 0, -1):
                lambdan[i] = sequence[i].p + max(sequence[i].js.q + sequence[i].js.p, lambdan[i + 1])
        elif(mov.type == "F"):
            for i in range(len(sequence) - 2, -1, -1):
                if(i == len(sequence) - 2):
                    lambdan[i] = sequence[i].p + max(sequence[i].js.q + sequence[i].js.p, sequence[-1].ms.q + sequence[-1].ms.p)
                else:
                    lambdan[i] = sequence[i].p + max(sequence[i].js.q + sequence[i].js.p, lambdan[i + 1])
            lambdan[-1] = sequence[-1].p + max(sequence[-1].js.q + sequence[-1].js.p, lambdan[0])
            lambda0[-1] = max(sequence[-1].jp.r, sequence[0].mp.r)
            lambda0[0] = max(sequence[0].jp.r, lambda0[-1])
            for i in range(len(sequence) - 2):
                lambda0[i + 1] = max(sequence[i + 1].jp.r, lambda0[i] + sequence[i].p)
        return max(np.asarray(lambda0) + np.asarray(lambdan))

def writeresult(data, sol, filename):
    outputfile = open(filename, 'w')
    test_id_resource_dict = copy.deepcopy(data.test_id_resource_dict)
    line = "Completion time:  " + str(f'{sol.cost:.2f}') + " \n"
    outputfile.write(line)
    outputs = [(o.j.jid, o.m.mid,o.starttime, o.testid) for o in sol.topoorder[1:-1]]
    outputs.sort(key = lambda x: x[2])
    output2 = []
    exhaustedset = set()
    for output in outputs:
        resource = test_id_resource_dict[output[3]]
        output2.append((output[0], output[1], round(output[2],2), resource))
        test_id_resource_dict[output[3]] = test_id_resource_dict[output[3]] - 1
    output2.sort(key = lambda x: (x[0], x[1])) 
    for output in output2:
        if output[3] <= 0:
            exhaustedset.add(output[0] + 1)
        line = "test {} starts on machine {} at time {:6.2f} - available resources: {:2d} \n".format(output[0] + 1, output[1] + 1, output[2], output[3] * (output[3] > 0))
        outputfile.write(line)
    outputfile.write("\n")
    for test in exhaustedset:
        line = "test resources exhausted for test # " + str(test) + "\n"
        outputfile.write(line)
    outputfile.close()    


def printsche(sol):
    scheduling = sol.sche
    for i in range(prob.m):
        for j in range(prob.n):
            print(str(scheduling[i][j]) + " ", end = " ")
        print("")
        

def printcriticalpath(sol):
     for bl in sol.criticalpath:
        print("[", end = "")
        for o in bl:
            print(str(o.oid) + " ", end = "")
        print("]", end = "")
        
def printtopoorder(sol):
    for oper in sol.topoorder[1:-1]:
        print(str(oper.oid) + " ", end = "")
    print("")


In [2]:
scheduling_filename = "scheduling_input.txt"
matrix_filename = "matrix_test.csv"
time_filename = "submission_task1_longhorn.csv"
output_filename = "submission_task2_longhorn.txt"

print("begin reading data")
data = readdata(scheduling_filename, matrix_filename, time_filename)
print("begin formulating problem")
prob = generateproblem(data)
print("begin generating initial solution")
sche = getinitialsche(prob)
print("begin solving the problem")
bestsol = getinitialsol(prob, sche)
print("Initial Solution: " + str(bestsol.cost))
for i in range(10):
    sol = optimize(prob, sche, "TS", "N5") 
    if not bestsol or bestsol.cost > sol.cost:
        bestsol = sol
    sol = optimize(prob, sche, "TS", "N6")    
    if not bestsol or bestsol.cost > sol.cost:
        bestsol = sol
    sol = optimize(prob, sche, "SA", "N5")    
    if not bestsol or bestsol.cost > sol.cost:
        bestsol = sol
    sol = optimize(prob, sche, "SA", "N6") 
    if not bestsol or bestsol.cost > sol.cost:
        bestsol = sol
print("begin writing output")
writeresult(data, bestsol, output_filename)
print('finish scheduling.')

begin reading data
begin formulating problem
begin generating initial solution
begin solving the problem
Initial Solution: 1261.0116176899999
TS&N5:979.90225431
TS&N6:1024.19853028
SA&N5:945.0776692699999
SA&N6:945.0776692699999
TS&N5:1036.32448779
TS&N6:1041.6088753499998
SA&N5:953.5135785399999
SA&N6:947.2438183699999
TS&N5:1036.32448779
TS&N6:1041.6088753499998
SA&N5:953.33659944
SA&N6:945.0776692699999
TS&N5:1131.97963347
TS&N6:1079.521244
SA&N5:963.2389412299999
SA&N6:962.22540853
TS&N5:1255.7553806799997
TS&N6:962.22540853
SA&N5:963.3851871299998
SA&N6:965.03240603
TS&N5:970.32262993
TS&N6:965.4050903299999
SA&N5:965.4050903299999
SA&N6:963.3851871299998
TS&N5:970.32262993
TS&N6:965.4050903299999
SA&N5:953.33659944
SA&N6:953.5135785399999
TS&N5:953.5135785399999
TS&N6:1047.46731576
SA&N5:962.22540853
SA&N6:962.22540853
TS&N5:970.32262993
TS&N6:970.32262993
SA&N5:962.22540853
SA&N6:965.4050903299999
TS&N5:970.32262993
TS&N6:970.32262993
SA&N5:953.5135785399999
SA&N6:945.0776692699